how-to for some common basic tasks when building hardware platforms.
    
    These guides use vivado 2023.1 and vitis 2023.1
    
    unless otherwise states these guides will use a PYNQ-Z1 board

    target language VHDL
    

# How to use Uartlite 
create block design, name it 'ic20_top'

[+] ZYNQ7 Processing System

[+] AXI UARTlite

    double click AXI Uartlite core and change baud rate to 115200

    run block automation 

    run connection automation (all automation)

    In 'Sources' right click ic20_top and create HDL wrapper. 

![pic](assets/uartlite_blockdiagram.png "Block Diagram")


![pic](assets/uartlite_sources.png "Sources")

click on Run Synthesis

after synthesis enter I/O planning perspective, from here under the I/O ports tab assign the uart ports

remember to change voltage to LVCMOS33 here.

![pic](assets/uartlite_io_ports.png "IO_PORTS")

ctrl-s to save, name your constraint something like "physical_constr.xdc"


note the constraint file that was created 

![pic](assets/uartlite_constraints.png "Constraints")

Generate Bitstream and export hardware.




------------------------------------------------


1. Open Vitis and create a workspace
2, create platform component using the .xsa we just exported (ic20_top_wrapper.xsa)
3.from example, create hello world appliction project.
4. edit hello_world.c
```
//hello_world.c
//baud rate: 115200
    #include <stdio.h>
#include "platform.h"
#include "xil_printf.h"
#include <xparameters.h>
#include <xuartlite.h>

#define uart_id XPAR_AXI_UARTLITE_0_BASEADDR

XUartLite uart;
int main()
{
    char msg[] = "Hello Daniel!\n\r";
    XUartLite_Initialize(&uart, uart_id);

    for(;;){
        XUartLite_Send(&uart,msg, sizeof(msg));
        sleep(1);
    }
    return 0;
}
```
XUartLite_Initialize  initializes axi_uartlite, it comes from the <xuartlite.h> library

build and run platform/application

|     PMOD A  |       |       |       |       |       |
|-------|-------|-------|-------|-------|-------|
|    0   |     GND  |    0   |     WHITE  | GREEN      |   0    |
|    0   |    0   |    0   |    0   |    0   |   0    |


open gtkterm to view output, remember to enable port in configuration.

![pic](assets/uartlite_output.png "Output")



# Creating a Shared BRAM

We want to create a BRAM that can be read and written to by both the hardcore and softcore.

![pic](assets/sharedBRAM_abstract.png "drawing of shared bram")

Create a new project 

Create block design

[+] ZYNQ7 Processing System 

Run block automation and choose defaults

[+] MicroBlaze

Run block automation, choose 64KB local memory and None for cache configuration.

[+] AXI BRAM controller

click on and rename this IP 'shared_bram_ctrl', double click and change from 2 ports to 1.

[+] Block Memory Generator

change name of this IP to 'shared_bram_0' and manually connect output of controller to input of shared_bram


![pic](assets/sharedBRAM_start.png "starting block diagram")

Run connection automation *twice*. selecting all automation.

Now, in Sources tab, drag and drop  "4 LEDS" and "4 buttons" in that order. this determines GPIO channel.

[+] AXI Uartlite

double click this IP and change baud rate to 115200

Run connection automation and reformat design.

![pic](assets/sharedBRAM_mid.png "middle")

open 'address editor' tab and notice that both the processing system and microblaze are on network 0

![pic](assets/sharedBRAM_address.png "shared address")

more specifically they both have shared_bram_ctrl at the same address.

there is another network.. it is for the microblaze local memory bus.

validate, Save, and create HDL wrapper.

create a constraint file (Alt-A) called 'uartlite_pmoda.xdc' 

add the following constraints

```
set_property PACKAGE_PIN Y19 [get_ports uart_rtl_rxd]
set_property IOSTANDARD LVCMOS33 [get_ports uart_rtl_rxd]
set_property PACKAGE_PIN Y16 [get_ports uart_rtl_txd]
set_property IOSTANDARD LVCMOS33 [get_ports uart_rtl_txd]
```

Generate Bitstream.

sidenote: the "number of jobs" is how many CPU cores the simulation is using. Increase this number at your own risk, it will speed things up.

Export hardware platform to vitis.

![pic](assets/sharedBRAM_done.png "shared address")

before we go to vitis, i would like to point out that our design would benefit from a custom rtl that chooses which BRAM to select.



Open a new workspace in Vitis, create a new hardware platform and import the platform we just created.












#  Intro to the Processing System

The processing system is what seperates our SoC from just any old fpga, The Zynq 7000 SoC that we are targeting comes outfitted with a dual-core ARM A9 processor. We often refer to this as the 'white core"

![PS](assets/PS_overview.png "overview")

![PS](assets/PS_ps_overview.png "overview")

In Vivado, create a block diagram.

[+] ZYNQ7 Processing System

connect Master clock to fabric clock

run block automation 

![PS](assets/PS_clock.png "overview")

double click the PS7 to get this overview, we can see that the built in uart is already activated. if it wasnt we could edit that here.

![PS](assets/PS_block.png "overview")

in Sources, right click ic20_top and create HDL wrapper. when this is complete, generate bitstream and export to vitis.

in Vitis, create a workspace and platform component using the bitstream we just exported from vivado.

![PS](assets/PS_platform.png "overview")

notice here we have the platform running on 1 core of the ARM processor.

Now, create an application from example, select 'hello_world' and edit the string to be printed in 'hello_world.c' under the sources directory.

open a gtkterm and select the appropriate port

build and run application in vitis to view the output

![PS](assets/PS_output.png "overview")

# how to use vitis python CLI

## after creating shared bram hw design with processing system and microblaze on a shared network.

in the same directory as your hw, open an interactive shell

import the required library

```import hsi```

using the hsi library import the xsa, ill call it HwDesign

``` HwDesign = hsi.HwManager.open_hw_design("ic20_top_wrapper.xsa")```

make a list of cells, filtering for processors

```procs = HwDesign.get_cells(hierarchical='true',filter='IP_TYPE==PROCESSOR')```

print list of processors in hw design 

```for procs in procs:```
    ```print(procs)```

![procs](assets/vitis_procs.png "processors")

from here we can inspect all the IP on the memory map of a specific processor

```memmap = HwDesign.get_mem_ranges(of_object=procs[0])```

notice that both microblaze and processing system can see the shared bram controller and gpio

![memmap](assets/vitis_memmap.png "memory map of mb vs ps ")





